# figure 1: methane over time since 2002

In [1]:
import pandas as pd
import plotly.express as px

# Read the CSV file into a DataFrame
df_county = pd.read_csv('/Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/Projekte/Wellcome Ideathon/methane_processing/final_csvs/methane_monthly_county.csv')

# Convert 'date' column to datetime data type
df_county['date'] = pd.to_datetime(df_county['date'])

FipsDF = pd.read_csv('/Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/Projekte/Wellcome Ideathon/methane_processing/data/fips2county.tsv', sep='\t', header='infer', dtype=str, encoding='latin-1')
StateAbbrDF = FipsDF[["StateAbbr", 'StateFIPS', "StateName"]].drop_duplicates()

df_county = pd.merge(df_county, StateAbbrDF.astype({'StateFIPS': 'int64'}), left_on='STATEFP', right_on='StateFIPS', how='left')
df_county['StateName'] = df_county['StateName'].astype('object')
df_county = df_county.dropna(subset=['StateName'])


In [2]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Convert 'Date' column to datetime data type
df_county['date'] = pd.to_datetime(df_county['date'])

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='county-dropdown',
        options=[{'label': StateName, 'value': StateName} for StateName in sorted(df_county['StateName'].unique())],
        value=None,
        placeholder='Select a state',
        style={'width': '200px', 'margin-bottom': '20px'}
    ),
    dcc.Graph(
        id='scatter-plot'
    )
])

# Define the callback function to update the scatter plot based on the dropdown selection
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('county-dropdown', 'value')]
)
def update_scatter_plot(selected_county):
    if selected_county is None:
        filtered_df = df_county
    else:
        filtered_df = df_county[df_county['StateName'] == selected_county]

    fig = px.scatter(filtered_df, x='date', y='ch4', color='ch4', trendline='lowess', range_color=[df_county['ch4'].min(), df_county['ch4'].max()])
    fig.update_traces(marker=dict(size=5))
    fig.update_layout(
        xaxis_title='Year',
        yaxis_title='Methane ppm',
        title='Rising methane emissions in the United States (2003-2021)',
        coloraxis_colorbar=dict(title='Methane ppb')
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/fw/sfmxzmg52fg2gpxsf9nk9x300000gn/T/ipykernel_30288/1049796691.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/fw/sfmxzmg52fg2gpxsf9nk9x300000gn/T/ipykernel_30288/1049796691.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# Figure 2: Methane emissions and daily slider

In [3]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from datetime import datetime

# Read the data
df_daily = pd.read_csv('/Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/Projekte/Wellcome Ideathon/methane_processing/final_csvs/methane_final_lonlat.csv')
states = gpd.read_file('/Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/Projekte/Wellcome Ideathon/methane_processing/data/gadm36_USA_shp/gadm36_USA_1.shp')

FipsDF = pd.read_csv('/Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/Projekte/Wellcome Ideathon/methane_processing/data/fips2county.tsv', sep='\t', header='infer', dtype=str, encoding='latin-1')
StateAbbrDF = FipsDF[["StateAbbr", 'StateFIPS', "StateName"]].drop_duplicates()

df_daily = pd.merge(df_daily, StateAbbrDF.astype({'StateFIPS': 'int64'}), left_on='STATEFP', right_on='StateFIPS', how='left')
df_daily['StateName'] = df_daily['StateName'].astype('object')
df_daily = df_daily.dropna(subset=['StateName'])

# Filter and sample the data
df_daily['date'] = pd.to_datetime(df_daily['date'])
df_daily['day_of_year'] = df_daily['date'].dt.dayofyear
df_daily['week_of_year'] = df_daily['date'].dt.isocalendar().week

df_filtered = df_daily[df_daily['date'].dt.year == 2019]


In [4]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px

# Create the Dash application
app = dash.Dash(__name__)

# Define the layout of the application
app.layout = html.Div([
    dcc.Graph(id='map-graph'),
    html.H3('Select week of the year:'),  # Add the title for the date slider
    dcc.Slider(
        id='date-slider',
        min=df_filtered['week_of_year'].min(),
        max=df_filtered['week_of_year'].max(),
        value=df_filtered['week_of_year'].min(),
        marks={str(week_of_year): str(week_of_year) for week_of_year in df_filtered['week_of_year'].unique()},
        step=None
    )
])

# Define the callback function to update the map based on the slider value
@app.callback(
    dash.dependencies.Output('map-graph', 'figure'),
    [dash.dependencies.Input('date-slider', 'value')]
)
def update_map(selected_date):
    filtered_data = df_filtered[df_filtered['week_of_year'] == selected_date]

    fig = px.scatter_mapbox(filtered_data, lat='latitude', lon='longitude',
                            hover_data=['ch4'], color='ch4', color_continuous_scale='Viridis',
                            range_color=[df_filtered['ch4'].min(), df_filtered['ch4'].max()],
                            size='ch4', size_max=10, zoom=3)

    fig.update_layout(mapbox_style='open-street-map')
    fig.update_traces(marker=dict(opacity=0.8))
    fig.update_layout(legend=dict(x=0, y=1), margin=dict(l=50, r=50, t=50, b=50))
    fig.update_layout(mapbox=dict(center=dict(lat=50, lon=-100), zoom=1.5))
    fig.update_layout(title='Methane emissions in the United States, 2019')

    return fig

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)


In [12]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px

# Create the Dash application
app = dash.Dash(__name__)

# Define the layout of the application
app.layout = html.Div([
    html.H3('Select state:'),  # Add the title for the dropdown menu
    dcc.Dropdown(
        id='state-dropdown',
        options=sorted([{'label': StateName, 'value': StateName} for StateName in df_filtered['StateName'].unique()], key=lambda x: x['label']),
        value=df_filtered['StateName'].unique()[0]
    ),
    dcc.Graph(id='map-graph'),
    html.H3('Select week of the year:'),  # Add the title for the week slider
    dcc.Slider(
        id='date-slider',
        min=df_filtered['week_of_year'].min(),
        max=df_filtered['week_of_year'].max(),
        value=df_filtered['week_of_year'].min(),
        marks={str(week_of_year): str(week_of_year) for week_of_year in df_filtered['week_of_year'].unique()},
        step=None
    )
])

# Define the callback function to update the map based on the slider value and selected state
@app.callback(
    dash.dependencies.Output('map-graph', 'figure'),
    [dash.dependencies.Input('date-slider', 'value'),
     dash.dependencies.Input('state-dropdown', 'value')]
)
def update_map(selected_date, selected_state):
    filtered_data = df_filtered[(df_filtered['week_of_year'] == selected_date) & (df_filtered['StateName'] == selected_state)]

    fig = px.scatter_mapbox(filtered_data, lat='latitude', lon='longitude',
                            hover_data=['ch4'], color='ch4', color_continuous_scale='Viridis',
                            range_color=[df_filtered['ch4'].min(), df_filtered['ch4'].max()],
                            size='ch4', size_max=10, zoom=3)

    fig.update_layout(mapbox_style='open-street-map')
    fig.update_traces(marker=dict(opacity=0.8))
    fig.update_layout(legend=dict(x=0, y=1), margin=dict(l=50, r=50, t=50, b=50))
    fig.update_layout(mapbox=dict(center=dict(lat=filtered_data['latitude'].mean(), lon=filtered_data['longitude'].mean()), zoom=4))
    fig.update_layout(title='Methane emissions in the United States, 2019')

    return fig

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)


## figure 3: methane deaths by race

In [36]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

methane_df = pd.read_csv('/Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/GitHub/Climate_Coders/data/health_dunja/Race_Methane.txt', delimiter='\t')

county_deaths = methane_df.groupby(['County', 'Single Race 6'])['Deaths'].sum().reset_index()

county_deaths['total_deaths'] = county_deaths.groupby('County')['Deaths'].transform('sum')

county_deaths['Proportion'] = county_deaths['Deaths'] / county_deaths['total_deaths']


,County,Single Race 6,Deaths,total_deaths,Proportion
0,"Abbeville County, SC",White,14.0,14.0,1.000000
1,"Acadia Parish, LA",White,36.0,36.0,1.000000
2,"Ada County, ID",White,145.0,145.0,1.000000
3,"Adair County, KY",White,13.0,13.0,1.000000
4,"Adams County, CO",Black or African American,22.0,213.0,0.103286


In [38]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Create the Dash application
app = dash.Dash(__name__)

# Define the layout of the application
app.layout = html.Div([
    dcc.Dropdown(
        id='county-dropdown',
        options=[{'label': county, 'value': county} for county in county_deaths['County']],
        value=county_deaths['County'][0]
    ),
    dcc.Graph(id='bar-plot')
])

# Define the callback function to update the bar plot based on the dropdown selection
@app.callback(
    Output('bar-plot', 'figure'),
    [Input('county-dropdown', 'value')]
)
def update_bar_plot(selected_county):
    filtered_data = county_deaths[county_deaths['County'] == selected_county]

    fig = px.bar(filtered_data, x='Single Race 6', y='Proportion', 
                 hover_data={'Single Race 6': False, 'Deaths': True},
                 labels={'Single Race 6': 'Race', 'Deaths': 'Number of Deaths'})

    fig.update_layout(
        title='Number of Deaths by Race in {}'.format(selected_county),
        xaxis_title='Race',
        yaxis_title='Number of Deaths'
    )

    return fig

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)


In [28]:

# Create the bar plot
fig = go.Figure(data=go.Bar(x=county_deaths['Single Race 6'], y=county_deaths['Deaths']))

# Customize the layout
fig.update_layout(
    title='Number of Deaths by Race',
    xaxis_title='Race',
    yaxis_title='Number of Deaths'
)

# Display the plot
fig.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Isolate just methane-related deaths here 
race_health_df_methane = race_health_df[race_health_df["ID"] == "methane"]

#### Filter by county if desired 
#county = "Calhoun County, AL"
#race_health_df_methane = race_health_df_methane[race_health_df_methane["County"] == county]

# Group the data by race and calculate the total deaths
race_deaths_agg = race_health_df_methane.groupby("Single Race 6")["Deaths"].sum().reset_index()

# Calculate the proportion of deaths for each race
total_deaths = race_deaths_agg["Deaths"].sum()
race_deaths_agg["Proportion of Deaths"] = race_deaths_agg["Deaths"] / total_deaths

# Create a bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Generate colors using the Viridis color palette
viridis_colors = cm.viridis(np.linspace(0, 1, len(race_deaths_agg)))

# Plot the bars with different colors for each race
ax.bar(
    race_deaths_agg["Single Race 6"],
    race_deaths_agg["Proportion of Deaths"],
    color=viridis_colors,
)

ax.set_title("Proportion of Methane-Related Deaths by Race")
ax.set_xlabel("Race")
ax.set_ylabel("Proportion of Deaths")
ax.set_xticklabels(race_deaths_agg["Single Race 6"], rotation=90)
ax.grid(True, linestyle="--", alpha=0.3)

# Add annotations for each bar
for i, row in race_deaths_agg.iterrows():
    proportion_text = f"{row['Proportion of Deaths']:.2%} of all deaths"
    ax.annotate(
        proportion_text,
        xy=(row["Single Race 6"], row["Proportion of Deaths"]),
        xytext=(0, 5),
        textcoords="offset points",
        ha="center",
        color="black",
    )

# Remove the top and right spines
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Adjust the layout
plt.tight_layout()

# Save the figure
#plt.savefig("proportion_of_deaths_race_BarChart.png")

# Display the plot
plt.show()